In [1]:
import os
import re
import pandas as pd
import numpy as np
from dplython import (DplyFrame, X, diamonds, select, sift, sample_n, sample_frac, arrange, mutate, group_by, summarize)

import plotly.plotly
import plotly.graph_objs as go
import pandas as pd

import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
from plotly import tools

%config IPCompleter.greedy=True

/anaconda3/envs/dev2/lib/python3.7/site-packages/socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
/anaconda3/envs/dev2/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__



In [2]:
in_data_path = '/Users/genie/data/oflc/kaggle/h1b_data_fy2011_fy2018_20190309.csv'
cities_data_path = '/Users/genie/data/oflc/uscitiesv1.3.csv'

In [ ]:
out_dir = '/Users/genie/dev/projects/github/h1b_data_analysis/out'
os.chdir(out_dir)

include_plotlyjs=True

In [3]:
df = pd.read_csv(in_data_path,quotechar='"',na_values='',encoding = "ISO-8859-1")
df = df.drop(['case_submitted','decision_date','full_time_position','soc_name','lat','lng'],axis=1)

/anaconda3/envs/dev2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



In [5]:
# fiscal year counts
df1 = DplyFrame(df) >> sift(X.case_status.notnull()) >> group_by(X.fiscal_year) >> summarize(fy_total=X.case_status.count())
df2 = DplyFrame(df) >> sift(X.case_status=="C") >> group_by(X.fiscal_year) >> summarize(fy_certified=X.case_status.count())
oflc_fy_counts = pd.merge(df1,df2,on='fiscal_year')
del df1
del df2

In [6]:
# fiscal year counts by state

df['state'] = df.apply(lambda x: x['work_state'] if pd.notnull(x['work_state']) else x['emp_state'] ,axis=1)

df1 = DplyFrame(df) >> sift(X.case_status.notnull(),X.state.notnull()) >> group_by(X.fiscal_year, X.state) >> summarize(fy_state_total=X.case_status.count())
df2 = DplyFrame(df) >> sift(X.case_status=="C",X.state.notnull()) >> group_by(X.fiscal_year, X.state) >> summarize(fy_certified_state=X.case_status.count())
df1 = pd.merge(df1,df2,on=['fiscal_year','state'])
del df2

oflc_fy_state_counts = pd.merge(df1, oflc_fy_counts, how='left', on='fiscal_year')
del df1

oflc_fy_state_counts['fy_state_total_share_pct'] =  oflc_fy_state_counts.apply(lambda x: round(x['fy_state_total']/x['fy_total'] * 100,2), axis=1)
oflc_fy_state_counts['fy_state_certified_share_pct'] =  oflc_fy_state_counts.apply(lambda x: round(x['fy_certified_state']/x['fy_certified'] * 100,2), axis=1)
oflc_fy_state_counts = oflc_fy_state_counts.drop(['fy_certified_state','fy_total','fy_certified'],axis=1)

zmin = oflc_fy_state_counts['fy_state_total'].min()
zmax = oflc_fy_state_counts['fy_state_total'].max()

scl = [[0.0, '#ffffff'],[0.2, '#ff9999'],[0.4, '#ff4d4d'], [0.6, '#ff1a1a'],[0.8, '#cc0000'],[1.0, '#4d0000']] # reds

# create data for slider
data_slider = []
for year in oflc_fy_state_counts.fiscal_year.unique():
    df_temp = oflc_fy_state_counts[(oflc_fy_state_counts['fiscal_year']== year)]
    
    for col in df_temp.columns:
        df_temp[col] = df_temp[col].astype(str)
    
    df_temp['text'] = 'FY' + df_temp['fiscal_year'] + ' Share (%) ' + df_temp['fy_state_total_share_pct'] 
    
    data_temp = dict(
        type='choropleth',
        colorscale = scl,
        zmin=zmin, 
        zmax=zmax,
        autocolorscale = False,
        locations = df_temp['state'],
        z = df_temp['fy_state_total'].astype(float),
        locationmode = 'USA-states',
        text = df_temp['text'],
        marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
        colorbar = dict(title = "Total applications"))
    
    data_slider.append(data_temp)

# create steps
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 2011)) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)


sliders = [dict(active=0, pad={"t": 1}, steps=steps)] 

layout = go.Layout(title=go.layout.Title(text='2011-2018 Statewide H-1B Application Volume<br>(Hover for breakdown)'),
                   geo=go.layout.Geo(scope='usa',projection=go.layout.geo.Projection(type='albers usa'),
                                     showlakes=True,lakecolor = 'rgb(255, 255, 255)'),
                   sliders=sliders)

fig = go.Figure(data = data_slider, layout = layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_statewide_dist.html', validate=True)

del oflc_fy_state_counts

In [7]:
# functions

def validateCity(x):
    x = re.sub("\n"," ", x)
    x = re.sub("\s","",x)
    return x.isalpha()

def deriveCityVariable(x):
    result = ''
    if(pd.notnull(x['work_city']) and validateCity(x['work_city'])==True):
        return x['work_city'].title()
    elif(pd.notnull(x['emp_city']) and validateCity(x['emp_city'])==True):
        return x['emp_city'].title()
    else:
        return None
    
df['city'] = df.apply(deriveCityVariable, axis=1)
df['city_state'] = df.apply(lambda x: ','.join([x['city'],x['state']]) if pd.notnull(x['city']) and pd.notnull(x['state']) else None,axis=1)

<>:3: DeprecationWarning:

invalid escape sequence \s



In [9]:
# fiscal year counts by city

df1 = DplyFrame(df) >> sift(X.case_status.notnull(),X.city_state.notnull()) >> group_by(X.fiscal_year, X.city_state) >> summarize(fy_city_total=X.case_status.count())

oflc_fy_city_counts = pd.merge(df1, oflc_fy_counts, how='left', on='fiscal_year')
del df1

oflc_fy_city_counts['fy_city_total_share_pct'] =  oflc_fy_city_counts.apply(lambda x: round(x['fy_city_total']/x['fy_total'] * 100,2), axis=1)
oflc_fy_city_counts = DplyFrame(oflc_fy_city_counts) >> sift(X.fy_city_total>50) >> select(X.fiscal_year,X.city_state,X.fy_city_total,X.fy_city_total_share_pct)
oflc_fy_city_counts['key'] = oflc_fy_city_counts['city_state'].apply(lambda x: x.upper() if x is not None else None) 

cities_df = pd.read_csv(cities_data_path,quotechar='"',na_values='',encoding = "ISO-8859-1")
cities_df['key'] = cities_df.apply(lambda x: x['city'] + ',' + x['state_id'], axis=1)
cities_df = cities_df.drop(['city','state_id','zip','county','Unnamed: 6'],axis=1)

oflc_fy_city_counts = pd.merge(oflc_fy_city_counts, cities_df, how='left', on='key')
del cities_df

oflc_fy_city_counts = oflc_fy_city_counts.drop(['key'],axis=1)
oflc_fy_city_counts = oflc_fy_city_counts.dropna()

x_min = float(oflc_fy_city_counts['fy_city_total'].min())
x_max = float(oflc_fy_city_counts['fy_city_total'].max())
norm_x = 10.0
norm_y = 50.0

oflc_fy_city_counts['size'] = oflc_fy_city_counts.apply(lambda x : round(norm_x + (norm_y-norm_x)*(x['fy_city_total']-x_min)/(x_max-x_min),0), axis=1)

scl = [[0.0,'#ffe5e5'],[0.1, '#ffcccc'], [0.2, '#ffb2b2'], [0.3, '#ff9999'],[0.4, '#ff7f7f'], [0.5, '#ff6666'], [0.6, '#ff4c4c'], [0.7, '#ff3232'], 
       [0.8, '#ff1919'], [0.9, '#ff0000'], [1.0, '#cc0000']] # reds

cmax=oflc_fy_city_counts['fy_city_total'].max()

# create data for slider
data_slider = []
for year in oflc_fy_city_counts.fiscal_year.unique():
    df_temp = oflc_fy_city_counts[(oflc_fy_city_counts['fiscal_year']== year)]
    
    for col in df_temp.columns:
        df_temp[col] = df_temp[col].astype(str)
    
    df_temp['text'] = df_temp['city_state'] + '<br>FY' + df_temp['fiscal_year'] + ' Total Applications:' + df_temp['fy_city_total'] + '<br>Share (%) ' + df_temp['fy_city_total_share_pct'] 
    
    data_temp = go.Scattergeo(
#         type='scattergeo',
        locationmode = 'USA-states',
        lon = df_temp['lng'].astype(float),
        lat = df_temp['lat'].astype(float),
        text = df_temp['text'],
        mode = 'markers',
        marker = dict( 
            size = df_temp['size'].astype(float), 
            opacity = 0.8,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            cmax = cmax,
            colorscale=scl,
            autocolorscale = False,
#             reversescale = True,
            color = df_temp['fy_city_total'].astype(float),
            colorbar=dict(
                title="H-1B distributions by city"
            )
        ))
    
    data_slider.append(data_temp)

# create steps
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 2011)) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)


sliders = [dict(active=0, pad={"t": 1}, steps=steps)] 

layout = dict(
        title = '2011-2018 H-1B distributions by city', 
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            showlakes=True,lakecolor = 'rgb(255, 255, 255)',
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5        
        ),
        sliders=sliders
    )

fig = go.Figure(data = data_slider, layout = layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_city_dist.html', validate=True)

In [13]:
# lets define some functions

def calc_emp_avg_annual_wage(x):
    result = None
    if(x['wage_from']>0 and x['wage_to']>0):
        result = round((x['wage_from'] + x['wage_to'])/2,2)
    else:
        result = x['wage_from']
    
    if(x['wage_unit']=="M"):
        result=round(result*12,2)
    elif(x['wage_unit']=="BW"):
        result=round(result*26,2)
    elif(x['wage_unit']=="W"):
        result = round(result*52,2)
    elif(x['wage_unit']=="H"):
        result = round(result*40*52,2)
    return(result)

def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])


In [14]:
selected_metros = ['Seattle', 'SF Bay Area', 'Dallas', 'Miami','Atlanta', 'Detroit-Ann Arbor', 'Boston', 'Los Angeles', 'New York-Newark',
       'Charlotte', 'Chicago', 'Tampa', 'Raleigh-Durham', 'Austin', 'Pittsburgh', 'San Diego','Washington DC', 'Philadelphia', 'Milwaukee', 
       'Phoenix', 'Houston','Denver', 'Minneapolis-St.Paul', 'St. Louis', 'Nashville', 'Orlando', 'Kansas City', 'Las Vegas','Sacremento', 
        'Richmond', 'Portland', 'Tulsa', 'Anchorage', 'Honolulu']

selected_metros = sorted(selected_metros)

oflc_2018_selected_metros = df[(df.fiscal_year==2018) & (df.case_status=="C") & (df.metro.isin(selected_metros))]
oflc_2018_selected_metros['avg_annual_wage'] = oflc_2018_selected_metros.apply(calc_emp_avg_annual_wage, axis=1)

In [16]:
## selected cities - salary plot

x_lables=selected_metros
y_labels=['P25','P50','P75','P90']

x_metro_data=[]
for x_metro in selected_metros:
    x_metro_values = oflc_2018_selected_metros[oflc_2018_selected_metros.metro==x_metro]['avg_annual_wage'].values
    q25, q50, q75, q90 = np.percentile(x_metro_values,[25,50,75,90])
    x_metro_data.append([round(q25,2),round(q50,2),round(q75,2),round(q90,2)])

traces = []
for ylabel_idx,y_label in enumerate(y_labels):
    ylabel_data_points = [x_metro_data[metro_idx][ylabel_idx] for metro_idx in range(len(x_metro_data))]
    traces.append(go.Bar(x=selected_metros,y=ylabel_data_points,name=y_labels[ylabel_idx]))
    
data = [traces[i] for i in range(len(traces))]

layout = go.Layout(barmode='stack', title='Percentile-Salary Distribution in Selected Cities (FY-2018)',yaxis=dict(title='Salary (Cumulative)'))

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_cities_percentile_salary_2018.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_cities_percentile_salary_2018.html'

In [17]:
## selected cities - outliers

x_lables=selected_metros
y_labels=['Low Income Segment <P25-1.5*IQR','Affluent Segment >P75+1.5*IQR']

x_metro_data=[]
y_labels_hover_text=[]
for x_metro in selected_metros:
    x_metro_values = oflc_2018_selected_metros[oflc_2018_selected_metros.metro==x_metro]['avg_annual_wage'].values
    q25, q50, q75, q90 = np.percentile(x_metro_values,[25,50,75,90])
    IQR = q75-q25
    l_outlier_wage = q25-1.5*IQR
    h_outlier_wage = q75+1.5*IQR
    l_outliers = list(filter(lambda x: x < l_outlier_wage ,x_metro_values))
    h_outliers = list(filter(lambda x: x > h_outlier_wage ,x_metro_values))
    x_metro_data.append([round(len(l_outliers)/len(x_metro_values)*100,2), round(len(h_outliers)/len(x_metro_values)*100,2)])
    y_labels_hover_text.append([human_format(l_outlier_wage),human_format(h_outlier_wage)])

traces = []
for ylabel_idx,y_label in enumerate(y_labels):
    ylabel_data_points = [x_metro_data[metro_idx][ylabel_idx] for metro_idx in range(len(x_metro_data))]
    ylabel_hover_text = [y_labels_hover_text[idx][ylabel_idx] for idx in range(len(y_labels_hover_text))]
    traces.append(go.Bar(x=selected_metros,y=ylabel_data_points,text=ylabel_hover_text, name=y_labels[ylabel_idx]))
    
data = [traces[i] for i in range(len(traces))]

layout = go.Layout(barmode='stack',title='Outlier Population Proportions in Selected Cities (FY-2018)',
                   yaxis=dict(title='% of H1B certified population'), legend=dict(x=0,y=1,traceorder='normal',orientation="h"))

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_cities_outliers_2018.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_cities_outliers_2018.html'

In [24]:
# Median salary in selected cities, 2011-2018

df1 = df[(df.case_status=="C") & (df.metro.isin(selected_metros))]
fiscal_years = sorted(df1['fiscal_year'].unique())
traces_a = []
for x_metro in selected_metros:
    y_values = []
    for year in fiscal_years:
        df_temp = df1[(df1.fiscal_year==year) & (df1.metro==x_metro)]
        df_temp['avg_annual_wage'] = df_temp.apply(calc_emp_avg_annual_wage,axis=1)
        q50, q90 = np.percentile(df_temp['avg_annual_wage'].values,[50,90])
        y_values.append(q50)
    traces_a.append(go.Scatter(x=fiscal_years,y=y_values, name=x_metro))

data = [traces_a[i] for i in range(len(traces_a))]
layout = go.Layout(title='Median Salary (2011-2018)',yaxis=dict(title='Median salary'), hovermode='closest')
fig = go.Figure(data,layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='test.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/test.html'